## Setup Feast
Create a sample `rbac` project with local storage.

In [1]:
!rm -rf rbac
!feast init rbac


Creating a new Feast repository in /Users/dmartino/projects/mlops/feast/rh-feast/examples/rbac-local/rbac.



Update the `feature_store.yaml` with an `auth` section derived from the Keycloak setup file [.env](.env)

In [25]:
!cat .env

OIDC_SERVER_URL='http://0.0.0.0:9999'
REALM='rbac_example'
CLIENT_ID='app'
CLIENT_SECRET='eFGTp6ZJ0tARKYPbdMKUZ1J8PJNWfXR8'
PASSWORD='password'


### Update the server YAML
Update the server YAML to use OIDC authorization

In [4]:
from dotenv import load_dotenv
import os
import yaml

def load_config_file(path):
    load_dotenv()

    with open(path, 'r') as file:
        config = yaml.safe_load(file) or {}
    return config

In [5]:
def update_config_with_auth(config):
    config['auth']['type']='oidc'
    config['auth']['auth_server_url']=os.getenv('OIDC_SERVER_URL')
    config['auth']['auth_discovery_url']=f"{os.getenv('OIDC_SERVER_URL')}/realms/{os.getenv('REALM')}/.well-known/openid-configuration"
    config['auth']['client_id']=os.getenv('CLIENT_ID')
    config['auth']['client_secret']=os.getenv('CLIENT_SECRET')
    config['auth']['realm']=os.getenv('REALM')
    config['auth']['username']=''
    config['auth']['password']='password'

In [6]:
def update_config_file(path):
    with open(path, 'w') as file:
        yaml.safe_dump(config, file, default_flow_style=False)

In [7]:
config = load_config_file('rbac/feature_repo/feature_store.yaml')
update_config_with_auth(config)
update_config_file('rbac/feature_repo/feature_store.yaml')

In [8]:
!cat rbac/feature_repo/feature_store.yaml

auth:
  auth_discovery_url: http://0.0.0.0:9999/realms/rbac_example/.well-known/openid-configuration
  auth_server_url: http://0.0.0.0:9999
  client_id: app
  client_secret: eFGTp6ZJ0tARKYPbdMKUZ1J8PJNWfXR8
  password: password
  realm: rbac_example
  type: oidc
  username: ''
entity_key_serialization_version: 2
online_store:
  path: data/online_store.db
  type: sqlite
project: rbac
provider: local
registry: data/registry.db


### Update the client YAML
Update the client YAML to use OIDC authorization

In [9]:
config = load_config_file('client/feature_store.yaml')
update_config_with_auth(config)
update_config_file('client/feature_store.yaml')

In [10]:
!cat client/feature_store.yaml

auth:
  auth_discovery_url: http://0.0.0.0:9999/realms/rbac_example/.well-known/openid-configuration
  auth_server_url: http://0.0.0.0:9999
  client_id: app
  client_secret: eFGTp6ZJ0tARKYPbdMKUZ1J8PJNWfXR8
  password: password
  realm: rbac_example
  type: oidc
  username: ''
entity_key_serialization_version: 2
offline_store:
  host: localhost
  port: 8815
  type: remote
online_store:
  path: http://localhost:6566
  type: remote
project: rbac
registry:
  path: localhost:6570
  registry_type: remote


### Apply the configuration

In [11]:
!feast -c rbac/feature_repo apply

/Users/dmartino/projects/mlops/feast/rh-feast/sdk/python/feast/feature_store.py:601: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
Created entity driver
Created feature view driver_hourly_stats
Created feature view driver_hourly_stats_fresh
Created on demand feature view transformed_conv_rate_fresh
Created on demand feature view transformed_conv_rate
Created feature service driver_activity_v2
Created feature service driver_activity_v1
Created feature service driver_activity_v3

Created sqlite table rbac_driver_hourly_stats_fresh
Created sqlite table rbac_driver_hourly_stats



### Validate permissions

There are no permissions after applying the example:

In [12]:
!feast -c rbac/feature_repo permissions list

NAME    TYPES    WITH_SUBCLASS    NAME_PATTERN    ACTIONS    ROLES


The `permissions check` command identifies the resources that have no permissions matching their type, name or tags.

In [13]:
!feast -c rbac/feature_repo permissions check

08/08/2024 05:20:22 PM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
The following resources are not secured by any permission configuration:
NAME                         TYPE
driver                       Entity
driver_hourly_stats          FeatureView
driver_hourly_stats_fresh    FeatureView
transformed_conv_rate_fresh  OnDemandFeatureView
transformed_conv_rate        OnDemandFeatureView
driver_hourly_stats          FeatureView
driver_hourly_stats_fresh    FeatureView
driver_activity_v2           FeatureService
driver_activity_v1           FeatureService
driver_activity_v3           FeatureService
driver_hourly_stats_source   FileSource
vals_to_add                  RequestSource
driver_stats_push_source     PushSource
The following actions are not secured by any permission configuration (Note: this might not be a security concern, depending on the u

### Applying permissions
Let's create some Permissions to cover basic scenarios.

First a simple permission to read the status of all the objects.

In [33]:
from feast import FeatureStore
from feast.feast_object import ALL_RESOURCE_TYPES
from feast.permissions.action import CRUD, QUERY, AuthzedAction, ALL_ACTIONS
from feast.permissions.permission import Permission
from feast.permissions.policy import RoleBasedPolicy

In [34]:
store = FeatureStore("rbac/feature_repo")

In [35]:
read_permission = Permission(
    name="read_permission",
    types=ALL_RESOURCE_TYPES,
    policy=RoleBasedPolicy(roles=["reader"]),
    actions=AuthzedAction.READ
)
store.registry.apply_permission(read_permission, store.project)

Now a specific permission to write online data (e.g. `materialize`) the `FeatureView`s whose name ends by `fresh`

In [36]:
from feast.feature_view import FeatureView
write_fresh_permission = Permission(
    name="write_fresh_permission",
    types=FeatureView,
    name_pattern=".*_fresh",
    policy=RoleBasedPolicy(roles=["fresh_writer"]),
    actions=AuthzedAction.WRITE_ONLINE
)
store.registry.apply_permission(write_fresh_permission, store.project)

Another one to match allow access to OFFLINE functions.

In [38]:
from feast.feature_view import FeatureView
from feast.feature_service import FeatureService
from feast.on_demand_feature_view import OnDemandFeatureView
offline_permission = Permission(
    name="offline_permission",
    types=[FeatureView, OnDemandFeatureView, FeatureService],
    policy=RoleBasedPolicy(roles=["batch_admin"]),
    actions= CRUD + [AuthzedAction.WRITE_OFFLINE, AuthzedAction.QUERY_OFFLINE]
)
store.registry.apply_permission(offline_permission, store.project)

Finally, ad `admin` permission to manage all the resources

In [39]:
admin_permission = Permission(
    name="admin_permission",
    types=ALL_RESOURCE_TYPES,
    policy=RoleBasedPolicy(roles=["store_admin"]),
    actions=ALL_ACTIONS
)
store.registry.apply_permission(admin_permission, store.project)

## Validate registered permissions

List all the permissions.

In [40]:
!feast -c rbac/feature_repo permissions list

NAME                    TYPES                WITH_SUBCLASS    NAME_PATTERN    ACTIONS        ROLES
read_permission         FeatureView          True                             READ           reader
                        OnDemandFeatureView
                        BatchFeatureView
                        StreamFeatureView
                        Entity
                        FeatureService
                        DataSource
                        ValidationReference
                        SavedDataset
                        Permission
write_fresh_permission  FeatureView          True             .*_fresh        WRITE_ONLINE   fresh_writer
offline_permission      FeatureView          True                             CREATE         batch_admin
                        OnDemandFeatureView                                   READ
                        FeatureService                                        UPDATE
                                                                          

List all the resources matching each configured permission.

In [41]:
!feast -c rbac/feature_repo permissions list -v

08/09/2024 10:10:26 AM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
08/09/2024 10:10:26 AM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
08/09/2024 10:10:26 AM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
08/09/2024 10:10:26 AM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.

The structure of the feast-permissions list --verbose command will be as in the following example:

For example: 

permissions
├── permission_1 ['role names list']
│   ├──

Describe one of the permissions.

In [28]:
!feast -c rbac/feature_repo permissions describe admin_permission

name: admin_permission
types:
- FEATURE_VIEW
- ON_DEMAND_FEATURE_VIEW
- BATCH_FEATURE_VIEW
- STREAM_FEATURE_VIEW
- ENTITY
- FEATURE_SERVICE
- DATA_SOURCE
- VALIDATION_REFERENCE
- SAVED_DATASET
- PERMISSION
withSubclasses: true
namePattern: ''
actions:
- CREATE
- READ
- UPDATE
- DELETE
- QUERY_ONLINE
- QUERY_OFFLINE
- WRITE_ONLINE
- WRITE_OFFLINE
policy:
  roleBasedPolicy:
    roles:
    - store_admin



List the roles specified by these permissions.

In [29]:
!feast -c rbac/feature_repo permissions list-roles

ROLE NAME
batch_admin
fresh_writer
reader
store_admin


For each configured role, list all the resources and operations that are allowed to a user impersonating this role.

In [30]:
!feast -c rbac/feature_repo permissions list-roles -v

08/08/2024 06:05:28 PM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
ROLE NAME     RESOURCE NAME                RESOURCE TYPE        PERMITTED ACTIONS
batch_admin   driver                       Entity               -
batch_admin   driver_hourly_stats          FeatureView          CREATE
                                                                READ
                                                                UPDATE
                                                                DELETE
                                                                QUERY_OFFLINE
                                                                WRITE_OFFLINE
batch_admin   driver_hourly_stats_fresh    FeatureView          CREATE
                                                                READ
                                                                UPD